In [1]:
%run common.ipynb

In [2]:
# Parameters
# Chose the variables to crate the model
input_cols = ['shape', 'profile_entrance', 'profile_exit', 'rising_entrance', 'rising_exit', 'shards', 'feathering', 'entrance_mounding', 'center_mounding', 'exit_mounding', 'mounding']
result_col = 'result'
train_dataset_file = 'DATA/Train/serration_prediction_train.xlsx'
test_dataset_file = 'DATA/Validation/serration_prediction_validation.xlsx'

In [3]:
# The data are displayed on a table
display_input_data(
    train_dataset_file,
    input_cols
)

,shape_E,shape_indeterminate,profile_entrance_V,profile_exit_V,rising_entrance_bilateral,rising_entrance_single,rising_exit_bilateral,rising_exit_single,shards_present,feathering_present,entrance_mounding_bilateral,entrance_mounding_single,center_mounding_bilateral,center_mounding_single,exit_mounding_bilateral,exit_mounding_single,mounding_marked,mounding_not marked
0,False,True,True,True,False,True,False,True,False,False,False,True,False,False,False,True,False,True
1,False,True,False,True,False,True,True,False,False,False,False,True,False,False,False,True,False,True
2,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
3,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
4,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,True,False,True,True,False,True,False,True,False,False,False,True,False,False,False,True,True,False
276,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
277,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
278,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [4]:
# The two datasets are tested for compatibility 
check_input_data_indices(train_dataset_file, test_dataset_file, input_cols)

Datasets are compatible


In [5]:
# Logistic regression model
model = train_model(
    train_dataset_file,
    input_cols,
    result_col
)
# model
print('b0 = ' + str(model.intercept_))
print('b1...n = ' + str(model.coef_))

b0 = [2.94177671]
b1...n = [[-1.21639648 -1.28412923 -0.23924146 -2.3325252  -1.73212203 -1.63566348
  -1.3639484  -0.16607839  1.6963423   1.68430953 -0.56727603 -0.18639544
  -0.30974718 -0.53760101 -0.37406523  1.41657166  0.46015146  0.21973993]]


In [6]:
# Test for overfitting
# Evaluation of the training
x_train, y_train = prepare_dataset(train_dataset_file, input_cols, result_col)
model.score(x_train, y_train)

0.9

In [7]:
# 11 variables were used 
evaluate_model(
    model,
    test_dataset_file,
    input_cols,
    result_col
)

0.7164179104477612

In [8]:
#evaluate by category/label

# Load data

df = pd.read_excel("DATA/Validation/serration_prediction_validation.xlsx")
label_df = df.groupby(by = "label")

#next step: sort by the number
# use documentation
# DataFrameGroupBy.get_group(name[, obj])

# Filter bad data, replace categories with dummies

x_df = df[input_cols].apply(lambda x: x.str.strip())
x = pd.get_dummies(x_df, drop_first=True)


y_df = df[result_col].str.strip()
y = pd.get_dummies(y_df, drop_first=True).iloc[:, 0]

label_df.groups

KeyError: 'label'

In [ ]:
# Model optimisation, search for the optimal parameter combination
# Printing score best and worst model
run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col)

In [10]:
import ipywidgets as widgets


# tex = "$$p = {\frac{1}{1 + e ^ {-(\beta_0 + \beta_1X_i + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \beta_5X_5)}}}$$"
# tex = "$$p = {\frac{1}{1 + e ^ {-(3 + 2X_1 + \beta_2X_2 + \beta_3X_3 + \beta_4X_4 + \beta_5X_5)}}}$$"
tex = r"$$p = {\frac{1}{1 + e ^ {-("

# Add intercept
tex += str(np.round(model.intercept_[0], decimals=2))

 
# Add coefficients
for i, b in enumerate(model.coef_[0]):
    if b < 0:
        sign = '-'
    elif b > 0:
        sign = '+'
    else:
        continue
    tex += f"{sign}{str(np.abs(np.round(b, decimals=2)))}x_{{{str(i+1)}}}"

tex += ")}}}$$"

widgets.HTMLMath(
    value=tex,
    placeholder='Logistic regression',
)

HTMLMath(value='$$p = {\\frac{1}{1 + e ^ {-(2.94-1.22x_{1}-1.28x_{2}-0.24x_{3}-2.33x_{4}-1.73x_{5}-1.64x_{6}-1…